Accessing the dataset

In [2]:
from urbaning import Dataset

# load the complete dataset
root_folder = "/thinkmate_ainin/THI_dataset/FINAL_Cooperative_Perception_Dataset_v0"
dataset = Dataset(root_folder)
# number of total sequences
print(len(dataset))

34


Accessing a sequence

In [3]:
# use indexing from the dataset
index = 10
# a Sequence instance
print(type(dataset[index]))
# or simple iterate over the dataset
for sequence in dataset:
    print(sequence.sequence_name)
    break

# you can also load one sequence manually
from urbaning.data import Sequence
# sequence to load
sequence_name = "20241127_0000_crossing1_00"
# load the sequence
sequence = Sequence(root_folder, sequence_name)

<class 'urbaning.data.sequence.Sequence'>
20241126_0024_crossing1_18


Accessing a frame

In [6]:
# use indexing from the sequence
index = 10
# a Frame instance
print(type(sequence[index]))
# or simple iterate over the sequence
for frame in sequence:
    # this frame timestamp
    print(frame.timestamp)
    break

<class 'urbaning.data.frame.Frame'>
1732711751.0


Accessing a infrastructure data

In [7]:
# iterate over the dictionary of infrastructures in the frame
for crossing_name in frame.infrastructures:
    # access the infrastructure data
    infrastructure = frame.infrastructures[crossing_name]
    # a Infrastructure instance
    print(type(infrastructure))
    break


<class 'urbaning.data.vehicle.Vehicle'>


Accessing a vehicle data

In [ ]:
# iterate over the dictionary of vehicles in the frame
for vehicle_name in frame.vehicles:
    # access the infrastructure data
    vehicle = frame.vehicles[vehicle_name]
    # a Vehicle instance
    print(type(vehicle))
    break

Accessing a camera data

In [ ]:
# iterate over the dictionary of cameras in the vehicle
for camera_name in vehicle.cameras:
    # access the